In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime

#from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
rawData = pd.read_csv('/Users/patrickadjei/Desktop/Western Folder/Project/London Hydro Data/Data/High School/Detail_Profile_All_2017-01-01_to_2017-12-31one.csv')
rawData['newTime'] = pd.to_datetime(rawData['Date/ Time']) #Changing the Date/ Time to time series from string

In [6]:
#This time series data have inconsistant intervals
#Therefore I am making the data hourly based by adding rows within the hour
dateDictionary = {}
for index, row in rawData.iterrows():
    if((str(row.newTime.date()) +' '+ str(row.newTime.hour) +':00') in dateDictionary):
        dateDictionary[(str(row.newTime.date()) +' '+ str(row.newTime.hour) +':00')] += row.kW
    else:
        dateDictionary[(str(row.newTime.date()) +' '+ str(row.newTime.hour) +':00')] = row.kW

#Dictionary 2 keeps count of all the occurances within the hour
dateDictionary2 = {}
for index, row in rawData.iterrows():
    if((str(row.newTime.date()) +' '+ str(row.newTime.hour) +':00') in dateDictionary2):
        dateDictionary2[(str(row.newTime.date()) +' '+ str(row.newTime.hour) +':00')] += 1
    else:
        dateDictionary2[(str(row.newTime.date()) +' '+ str(row.newTime.hour) +':00')] = 1



In [8]:
newkWhValue=[]
newTimeSeries = []
newTimeStamp = []
for i in range(len(dateDictionary)):
    newkWhValue.append(list(dateDictionary.values())[i]/ list(dateDictionary2.values())[i]) #Taking the avg
    newTimeSeries.append(pd.to_datetime(list(dateDictionary.keys())[i])) #getting the generated keys
    newTimeStamp.append(pd.Timestamp(list(dateDictionary.keys())[i] +':00').timestamp()) #getting the unix timestamp representation

In [14]:
#Creating a dataframe
df = pd.DataFrame({'Time':newTimeSeries, 'Values':newkWhValue})

In [12]:
#Feature engineering part of the day which are: late night, early morning, morning, noon, evening night

#Theorectically, the recurrent neural network do not allow for remembering longter state. there the average is taken over the years to reduce remembering states
dayPart = []
for index,row in df.iterrows():
    if(row.Time.hour >= 0 and row.Time.hour < 4):
        dayPart.append('late night') #'late night'
        #print(row.Time.hour)
    elif(row.Time.hour > 3 and row.Time.hour < 8):
        dayPart.append('early morning') #'early morning'
        #print(row.Time.hour)
    elif(row.Time.hour > 7 and row.Time.hour < 12):
        dayPart.append('morning') #'morning'
        #print(row.Time.hour)
    elif(row.Time.hour > 11 and row.Time.hour < 16):
        dayPart.append('noon') #'noon'
        #print(row.Time.hour)
    elif(row.Time.hour > 15 and row.Time.hour < 20):
        dayPart.append('evening') #'evening'
        #print(row.Time.hour)
    elif(row.Time.hour > 19 and row.Time.hour <= 23):
        dayPart.append('night') #'night'
        
    
#Feature engineering type of day which are  either weekend or weekday
typeOfDay = []
for index,row in df.iterrows():
    if(row.Time.dayofweek == 5 or row.Time.dayofweek == 6):
        typeOfDay.append(0) #Weekend
    else:
        typeOfDay.append(1) #Weekday
        

        
#Normalizing the time stamps
norm1 = newTimeStamp[:3000] #This is for the training data
norm2 = newTimeStamp[3000:] #This is for the testing data       

norm1 = (norm1 - np.min(norm1))/ ( np.max(norm1) - np.min(norm1)) #This is for the training data
norm2 = (norm2 - np.min(norm2))/ ( np.max(norm2) - np.min(norm2)) #This is for the testing data 

norms_result = []
for i in range(len(norm1)):
    norms_result.append(norm1[i])
#adding the timestamps back together
for i in range(len(norm2)):
    norms_result.append(norm2[i])
    
#adding the normalized to the df along with the encoded representation of the daypart
df = pd.DataFrame({'Time':norms_result,'TypeOfDay':typeOfDay})
oneHot = pd.get_dummies(dayPart)
df = df.join(oneHot)
    
    
#Normalizing the target values
norm1 = newkWhValue[:3000] #This is for the training data
norm2 = newkWhValue[3000:] #This is for the testing data       

norm1 = (norm1 - np.min(norm1))/ ( np.max(norm1) - np.min(norm1)) #This is for the training data
norm2 = (norm2 - np.min(norm2))/ ( np.max(norm2) - np.min(norm2)) #This is for the testing data 

#adding the target values back together
norms_result = []
for i in range(len(norm1)):
    norms_result.append(norm1[i])
#adding the timestamps back together
for i in range(len(norm2)):
    norms_result.append(norm2[i])

#******* Algorithm handles sequencial data talk about validating the sequence

In [7]:
from keras.layers import Input, merge, Embedding, SimpleRNN
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [ ]:
#initial Tuning Parameter
LSTMfirstLayerNeurons = [10,20,60,100,200]
LSTMSecondLayerNeurons = [5,10,20,40,100]
LSTMThirdLayerNeurons = [2,5,10,20,40]
LSTMFourthLayerNeurons = [2,3,8,15,22]
LearningRates = [0.1,0.06,0.03,0.01]
Epochs = [10,10,15,20,30]

results = []

In [9]:
#ploting results after each iteration(Epoch)
model_histories = []
index = 0;

data_Matrix = list(df.values)
data_Matrix = np.reshape(data_Matrix,(len(data_Matrix),1,8))
target_Vector = np.array(norms_result)
target_Vector = target_Vector.reshape((len(target_Vector),1))

X_Train = data_Matrix[:3000]
Y_Train = target_Vector[:3000]

X_Test = data_Matrix[3000:]
Y_Test = target_Vector[3000:]

for a in range(len(LSTMfirstLayerNeurons)):
    model = Sequential()
    model.add(LSTM(LSTMfirstLayerNeurons[a], input_shape=(X_Train.shape[1:]),return_sequences=True,activation=act))
    model.add(Dropout(0.2))
    model.add(LSTM(LSTMSecondLayerNeurons[a],activation=act,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(LSTMThirdLayerNeurons[a],activation=act,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(LSTMFourthLayerNeurons[a],activation=act))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation=act))
    for b in range(len(LearningRates)):
        index+=1
        sgd = optimizers.Adam(lr=LearningRates[b])
        
        model.compile(loss='mean_squared_error', optimizer=sgd ,metrics=['accuracy'])
        model_history = model.fit(X_Train, Y_Train,epochs=Epochs[0], verbose=0, validation_data=(X_Test,Y_Test), shuffle=False)
        model_histories.append(model_history)
        #predictTrain = model.predict(X_Train)
        predictTest = model.predict(X_Test)
        #reversing the normalization of the output
        invNormY_Test =  Y_Test * (np.max(newkWhValue[3000:]) - np.min(newkWhValue[3000:]) +  np.min(newkWhValue[3000:])) #This is for the testing data 
        invNormPred_Test = predictTest * (np.max(newkWhValue[3000:]) - np.min(newkWhValue[3000:]) +  np.min(newkWhValue[3000:]))
        
        results.append(['Adam','4_layers',LSTMfirstLayerNeurons[a],LSTMSecondLayerNeurons[a],LSTMThirdLayerNeurons[a],LSTMFourthLayerNeurons[a],LearningRates[b],Epochs[0],mean_absolute_error(invNormY_Test, invNormPred_Test),mean_squared_error(invNormY_Test, invNormPred_Test)])



Train on 3000 samples, validate on 1381 samples
Epoch 1/100
 - 2s - loss: 0.1674 - acc: 3.3333e-04 - val_loss: 0.1494 - val_acc: 7.2411e-04
Epoch 2/100
 - 0s - loss: 0.1405 - acc: 3.3333e-04 - val_loss: 0.1473 - val_acc: 7.2411e-04
Epoch 3/100
 - 0s - loss: 0.1291 - acc: 3.3333e-04 - val_loss: 0.1243 - val_acc: 7.2411e-04
Epoch 4/100
 - 0s - loss: 0.1138 - acc: 3.3333e-04 - val_loss: 0.1106 - val_acc: 7.2411e-04
Epoch 5/100
 - 0s - loss: 0.0982 - acc: 3.3333e-04 - val_loss: 0.1001 - val_acc: 7.2411e-04
Epoch 6/100
 - 0s - loss: 0.0862 - acc: 3.3333e-04 - val_loss: 0.0947 - val_acc: 7.2411e-04
Epoch 7/100
 - 0s - loss: 0.0789 - acc: 3.3333e-04 - val_loss: 0.0934 - val_acc: 7.2411e-04
Epoch 8/100
 - 0s - loss: 0.0756 - acc: 3.3333e-04 - val_loss: 0.0927 - val_acc: 7.2411e-04
Epoch 9/100
 - 0s - loss: 0.0719 - acc: 3.3333e-04 - val_loss: 0.0904 - val_acc: 7.2411e-04
Epoch 10/100
 - 0s - loss: 0.0691 - acc: 3.3333e-04 - val_loss: 0.0902 - val_acc: 7.2411e-04
Epoch 11/100
 - 0s - loss: 0.06

 - 0s - loss: 0.0638 - acc: 6.6667e-04 - val_loss: 0.0944 - val_acc: 0.0014
Epoch 93/100
 - 0s - loss: 0.0639 - acc: 3.3333e-04 - val_loss: 0.0949 - val_acc: 0.0014
Epoch 94/100
 - 0s - loss: 0.0637 - acc: 3.3333e-04 - val_loss: 0.0943 - val_acc: 0.0014
Epoch 95/100
 - 0s - loss: 0.0637 - acc: 6.6667e-04 - val_loss: 0.0952 - val_acc: 0.0014
Epoch 96/100
 - 0s - loss: 0.0636 - acc: 6.6667e-04 - val_loss: 0.0952 - val_acc: 0.0014
Epoch 97/100
 - 0s - loss: 0.0637 - acc: 6.6667e-04 - val_loss: 0.0955 - val_acc: 0.0014
Epoch 98/100
 - 0s - loss: 0.0639 - acc: 6.6667e-04 - val_loss: 0.0993 - val_acc: 0.0014
Epoch 99/100
 - 0s - loss: 0.0638 - acc: 6.6667e-04 - val_loss: 0.0945 - val_acc: 0.0014
Epoch 100/100
 - 0s - loss: 0.0638 - acc: 6.6667e-04 - val_loss: 0.0949 - val_acc: 0.0014


In [204]:
#New tuning Parameter
LSTMfirstLayerNeurons = [8,9,10,11,12]
LSTMSecondLayerNeurons = [2,3,5,7,9]
LSTMThirdLayerNeurons = [1,2,2,2,3]
LSTMFourthLayerNeurons = [1,2,2,2,3]
LearningRates = [0.1,0.06,0.03,0.01]
Epochs = [10]

results = []

(1, 8)

In [ ]:
#poting results after each iteration(Epoch)
model_histories = []
index = 0;

data_Matrix = list(df.values)
data_Matrix = np.reshape(data_Matrix,(len(data_Matrix),1,8))
target_Vector = np.array(norms_result)
target_Vector = target_Vector.reshape((len(target_Vector),1))

X_Train = data_Matrix[:3000]
Y_Train = target_Vector[:3000]

X_Test = data_Matrix[3000:]
Y_Test = target_Vector[3000:]

for a in range(len(LSTMfirstLayerNeurons)):
    model = Sequential()
    model.add(LSTM(LSTMfirstLayerNeurons[a], input_shape=(X_Train.shape[1:]),return_sequences=True,activation=act))
    model.add(Dropout(0.2))
    model.add(LSTM(LSTMSecondLayerNeurons[a],activation=act,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(LSTMThirdLayerNeurons[a],activation=act,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(LSTMFourthLayerNeurons[a],activation=act))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation=act))
    for b in range(len(LearningRates)):
        index+=1
        sgd = optimizers.Adam(lr=LearningRates[b])
        
        model.compile(loss='mean_squared_error', optimizer=sgd ,metrics=['accuracy'])
        model_history = model.fit(X_Train, Y_Train,epochs=Epochs[0], verbose=0, validation_data=(X_Test,Y_Test), shuffle=False)
        model_histories.append(model_history)
        #predictTrain = model.predict(X_Train)
        predictTest = model.predict(X_Test)
        #reversing the normalization of the output
        invNormY_Test =  Y_Test * (np.max(newkWhValue[3000:]) - np.min(newkWhValue[3000:]) +  np.min(newkWhValue[3000:])) #This is for the testing data 
        invNormPred_Test = predictTest * (np.max(newkWhValue[3000:]) - np.min(newkWhValue[3000:]) +  np.min(newkWhValue[3000:]))
        
        results.append(['Adam','4_layers',LSTMfirstLayerNeurons[a],LSTMSecondLayerNeurons[a],LSTMThirdLayerNeurons[a],LSTMFourthLayerNeurons[a],LearningRates[b],Epochs[0],mean_absolute_error(invNormY_Test, invNormPred_Test),mean_squared_error(invNormY_Test, invNormPred_Test)])
        #ploting the figures
        plt.figure()
        plt.plot(model_history.history['loss'])
        plt.plot(model_history.history['val_loss'])
        plt.title('Model loss'+str(index))
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.show()
        
        

In [10]:
      
for i in range(len(results)):
    print(results[i])

In [ ]:
#Showing the actual vs the predicted plot

aa=[x for x in range(len(X_Test))]
plt.figure(figsize=(8,4))
plt.plot(aa, invNormY_Test, marker='.', label="actual")
plt.plot(aa, invNormPred_Test[:,0][:len(X_Test)], 'r', label="prediction")

plt.tight_layout()
#sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();